# Smart Flashcard Backend

FastAPI-based backend to store flashcards with subject inference.

In [ ]:
!pip install fastapi uvicorn sqlite3
import sqlite3
import random
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List

### Subject Inference Logic

In [ ]:
def infer_subject(text):
    text = text.lower()
    rules = {
        "photosynthesis": "Biology",
        "mitochondria": "Biology",
        "evolution": "Biology",
        "atom": "Chemistry",
        "molecule": "Chemistry",
        "reaction": "Chemistry",
        "law": "Physics",
        "force": "Physics",
        "acceleration": "Physics",
        "theorem": "Mathematics",
        "equation": "Mathematics",
        "derivative": "Mathematics"
    }
    for keyword, subject in rules.items():
        if keyword in text:
            return subject
    return "General"

### Database Initialization and Functions

In [ ]:
def init_db():
    conn = sqlite3.connect("flashcards.db")
    c = conn.cursor()
    c.execute("""CREATE TABLE IF NOT EXISTS flashcards (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    student_id TEXT,
                    question TEXT,
                    answer TEXT,
                    subject TEXT
                )""")
    conn.commit()
    conn.close()

init_db()

In [ ]:
def add_flashcard_to_db(student_id, question, answer, subject):
    conn = sqlite3.connect("flashcards.db")
    c = conn.cursor()
    c.execute("INSERT INTO flashcards (student_id, question, answer, subject) VALUES (?, ?, ?, ?)",
              (student_id, question, answer, subject))
    conn.commit()
    conn.close()

In [ ]:
def get_mixed_flashcards(student_id, limit):
    conn = sqlite3.connect("flashcards.db")
    c = conn.cursor()
    c.execute("SELECT question, answer, subject FROM flashcards WHERE student_id = ?", (student_id,))
    rows = c.fetchall()
    conn.close()

    subject_map = {}
    for row in rows:
        subject = row[2]
        subject_map.setdefault(subject, []).append({
            "question": row[0],
            "answer": row[1],
            "subject": subject
        })

    mixed_flashcards = []
    while subject_map and len(mixed_flashcards) < limit:
        for subject in list(subject_map):
            if subject_map[subject]:
                mixed_flashcards.append(subject_map[subject].pop(0))
                if len(mixed_flashcards) == limit:
                    break
            else:
                del subject_map[subject]

    random.shuffle(mixed_flashcards)
    return mixed_flashcards

### FastAPI Endpoints

In [ ]:
app = FastAPI()

class FlashcardRequest(BaseModel):
    student_id: str
    question: str
    answer: str

class FlashcardResponse(BaseModel):
    message: str
    subject: str

@app.post("/flashcard", response_model=FlashcardResponse)
def add_flashcard(flashcard: FlashcardRequest):
    subject = infer_subject(flashcard.question)
    add_flashcard_to_db(flashcard.student_id, flashcard.question, flashcard.answer, subject)
    return {"message": "Flashcard added successfully", "subject": subject}

@app.get("/get-subject")
def get_flashcards(student_id: str, limit: int = 5):
    flashcards = get_mixed_flashcards(student_id, limit)
    return flashcards